<a href="https://colab.research.google.com/github/kingketan9/DeepLearningLabs/blob/main/Ass8a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
input_size = 9 # number of features to extract (e.g., number of data channels)
hidden_size = 16 # number of units in the hidden state
num_layers = 1 # number of vertical stacks of hidden layers (note: only the final layer gives an output)
actfun = 'tanh'
bias = True

In [3]:
rnn = nn.RNN(input_size,hidden_size,num_layers,nonlinearity=actfun,bias=bias)
print(rnn)

RNN(9, 16)


In [4]:
seqlength = 5
batchsize = 2

In [5]:
X = torch.rand(seqlength,batchsize,input_size)
X.shape

torch.Size([5, 2, 9])

In [6]:
hidden = torch.zeros(num_layers,batchsize,hidden_size)

In [7]:
y,h = rnn(X,hidden)
print(f' Input shape: {list(X.shape)}')
print(f'Hidden shape: {list(h.shape)}')
print(f'Output shape: {list(y.shape)}')

 Input shape: [5, 2, 9]
Hidden shape: [1, 2, 16]
Output shape: [5, 2, 16]


In [8]:
y,h1 = rnn(X,hidden)
print(h1), print('\n\n')

tensor([[[-0.2519,  0.3668,  0.1052, -0.1049,  0.0742,  0.0783,  0.5663,
          -0.3682, -0.0759, -0.1252,  0.4505,  0.1601, -0.4417,  0.0543,
           0.6245,  0.3496],
         [-0.2218,  0.3268, -0.2336,  0.0684,  0.0224,  0.1236,  0.4955,
          -0.4589, -0.1727,  0.2756,  0.6374,  0.0821, -0.5239, -0.1229,
           0.5849,  0.3933]]], grad_fn=<StackBackward0>)





(None, None)

In [9]:
y,h2 = rnn(X)
print(h2), print('\n\n')

tensor([[[-0.2519,  0.3668,  0.1052, -0.1049,  0.0742,  0.0783,  0.5663,
          -0.3682, -0.0759, -0.1252,  0.4505,  0.1601, -0.4417,  0.0543,
           0.6245,  0.3496],
         [-0.2218,  0.3268, -0.2336,  0.0684,  0.0224,  0.1236,  0.4955,
          -0.4589, -0.1727,  0.2756,  0.6374,  0.0821, -0.5239, -0.1229,
           0.5849,  0.3933]]], grad_fn=<StackBackward0>)





(None, None)

In [10]:
print(h1-h2)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       grad_fn=<SubBackward0>)


In [11]:
for p in rnn.named_parameters():
    if 'weight' in p[0]:
        print(f'{p[0]} has size {list(p[1].shape)}')

weight_ih_l0 has size [16, 9]
weight_hh_l0 has size [16, 16]


In [12]:
class RNNnet(nn.Module):
    def __init__(self,input_size,num_hidden,num_layers):
        super().__init__()
        # store parameters
        self.input_size = input_size
        self.num_hidden = num_hidden
        self.num_layers = num_layers
        # RNN Layer
        self.rnn = nn.RNN(input_size,num_hidden,num_layers)
        # linear layer for output
        self.out = nn.Linear(num_hidden,1)
    
    def forward(self,x):
        print(f'Input: {list(x.shape)}')
        # initialize hidden state for first input
        hidden = torch.zeros(self.num_layers,batchsize,self.num_hidden)
        print(f'Hidden: {list(hidden.shape)}')
        # run through the RNN layer
        y,hidden = self.rnn(x,hidden)
        print(f'RNN-out: {list(y.shape)}')
        print(f'RNN-hidden: {list(hidden.shape)}')
        # pass the RNN output through the linear output layer
        o = self.out(y)
        print(f'Output: {list(o.shape)}')
        return o,hidden
    


In [13]:
net = RNNnet(input_size,hidden_size,num_layers)
print(net), print(' ')

for p in net.named_parameters():
    print(f'{p[0]} has size {list(p[1].shape)}')

RNNnet(
  (rnn): RNN(9, 16)
  (out): Linear(in_features=16, out_features=1, bias=True)
)
 
rnn.weight_ih_l0 has size [16, 9]
rnn.weight_hh_l0 has size [16, 16]
rnn.bias_ih_l0 has size [16]
rnn.bias_hh_l0 has size [16]
out.weight has size [1, 16]
out.bias has size [1]


In [14]:
X = torch.rand(seqlength,batchsize,input_size)
y = torch.rand(seqlength,batchsize,1)
yHat,h = net(X)

Input: [5, 2, 9]
Hidden: [1, 2, 16]
RNN-out: [5, 2, 16]
RNN-hidden: [1, 2, 16]
Output: [5, 2, 1]


In [15]:
lossfun = nn.MSELoss()
lossfun(yHat,y)

tensor(0.1959, grad_fn=<MseLossBackward0>)